In [1]:
import numpy as np
import pandas as pd

In [2]:

df = pd.read_excel("data\\data.xlsx")

In [3]:
nan_counts = df.isna().sum()
nan_counts

Ngay                                        0
Gio                                         0
Luong mua                                2547
Trượt lở (0-Không; 1-Xảy ra trượt lở)       0
dtype: int64

Nếu dữ liệu cùng ngày thì nan được thay bằng trung bình lượng mưa 3 giờ liên tiếp cùng ngày

In [4]:
mua = df['Luong mua'].copy()

In [5]:
for i in range(len(mua)):
    if pd.isna(mua[i]):
        temp= [mua[i-1],mua[i-2],mua[i-3]]
        mua[i]=np.nanmean(temp)

df['Luong mua']=mua
df = df.rename(columns={"Trượt lở (0-Không; 1-Xảy ra trượt lở)": "truot_lo"})


In [6]:
nan_counts = df.isna().sum()
nan_counts

Ngay         0
Gio          0
Luong mua    0
truot_lo     0
dtype: int64

In [7]:
df.to_csv('data\\data.csv', index=False)  

In [8]:
df['truot_lo'].value_counts()


truot_lo
0    87551
1      121
Name: count, dtype: int64

Mất cân bằng dữ liệu. Xử lý dự đoán bằng tính thống kê.


Mức độ sạt lở được đánh giá bằng xác suất sạt lờ tại giờ,ngày tháng đó và tương quan lượng mưa so với lượng mưa sạt lở thấp nhất tại giờ đó 

In [9]:
def chuan_bi_data():
    df['Ngay'] = pd.to_datetime(df['Ngay'], format='%Y-%m-%d', errors='coerce')
    df['month'] = df['Ngay'].dt.month
    df['day'] = df['Ngay'].dt.day
    df['hour'] = df['Gio']
    df['thoi_diem'] = df.apply(
        lambda row: f"{int(row['month']):02d}-{int(row['day']):02d} {int(row['hour']):02d}h",
        axis=1
    )
    df_sl = df[df['truot_lo'] == 1]
    mua_min = df_sl['Luong mua'].min()
    mua_tb = df_sl.groupby('thoi_diem')['Luong mua'].mean().reset_index(name='luong_mua_tb_khi_satlo')
    xac_suat = df.groupby('thoi_diem')['truot_lo'].mean().reset_index(name='xac_suat_satlo')
    tk = xac_suat.merge(mua_tb, on='thoi_diem', how='left') 
    tk = tk.fillna(0)
    tk['xac_suat_satlo'] = (tk['xac_suat_satlo'] > 0).astype(int)
    tk['luong_mua_min_satlo'] = mua_min
    tk.to_csv("data\\du_doan.csv", index=False)


In [10]:
chuan_bi_data()

In [ ]:

def du_doan(gio, ngay, thang, luong_mua, file_path='data\du_doan.csv'):
    thoi_diem = f"{int(thang):02d}-{int(ngay):02d} {int(gio):02d}h"
    try:
        df = pd.read_csv(file_path)
        row = df[df['thoi_diem'] == thoi_diem]
        if row.empty:
            return f"Khong"
        xac_suat = row['xac_suat_satlo'].values[0]
        mua_tb = row['luong_mua_tb_khi_satlo'].values[0]
        mua_min = row['luong_mua_min_satlo'].iloc[0]
        
        if luong_mua >= mua_min :
            return f'Cao'
        if xac_suat == 0:
            return f"Thap"

        if luong_mua >= mua_tb:
            return f"Cao"
        else:
            return f"Thap"

    except FileNotFoundError:
        return 
    
du_doan(21,8,7,15)


'Cao.'